# Classification & Summarization Task Based On BERT and Pretrained Model

Data preparing

In [1]:
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json
import string
import gzip

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

import numpy as np

import os
import re
import random
from summarizer import Summarizer
import random

In [4]:
f = gzip.open('/content/drive/MyDrive/CSE256_summary/cs_dataset/renttherunway_final_data.json.gz')
ori_dataset = []
cnt=0
for l in f:
  ori_dataset.append(json.loads(l))
  cnt+=1


Classification

In [ ]:
import collections
thedic=collections.defaultdict(int)
for data in ori_dataset:
  thedic[data['category']]+=1

thedic

defaultdict(int,
            {'romper': 3070,
             'gown': 44381,
             'sheath': 19316,
             'dress': 92884,
             'leggings': 112,
             'top': 4931,
             'jumpsuit': 5184,
             'sweater': 1149,
             'jacket': 2404,
             'shirtdress': 729,
             'maxi': 3443,
             'shift': 5365,
             'pants': 422,
             'shirt': 277,
             'mini': 1751,
             'skirt': 1531,
             'pullover': 58,
             'blouse': 651,
             'suit': 123,
             'coat': 980,
             'trench': 20,
             'bomber': 128,
             'cape': 84,
             'blazer': 782,
             'vest': 278,
             'duster': 12,
             'ballgown': 16,
             'tank': 181,
             'poncho': 48,
             'frock': 205,
             'tunic': 162,
             'cardigan': 241,
             'culottes': 188,
             'down': 464,
             'trouser': 55,
     

In [ ]:
ori_dataset[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [ ]:
label_map={}
cnt=0
for data in thedic:
  #if thedic[data]>20 and thedic[data]<300:
    label_map[data]=cnt
    cnt+=1
label_map

{'romper': 0,
 'gown': 1,
 'sheath': 2,
 'dress': 3,
 'leggings': 4,
 'top': 5,
 'jumpsuit': 6,
 'sweater': 7,
 'jacket': 8,
 'shirtdress': 9,
 'maxi': 10,
 'shift': 11,
 'pants': 12,
 'shirt': 13,
 'mini': 14,
 'skirt': 15,
 'pullover': 16,
 'blouse': 17,
 'suit': 18,
 'coat': 19,
 'trench': 20,
 'bomber': 21,
 'cape': 22,
 'blazer': 23,
 'vest': 24,
 'duster': 25,
 'ballgown': 26,
 'tank': 27,
 'poncho': 28,
 'frock': 29,
 'tunic': 30,
 'cardigan': 31,
 'culottes': 32,
 'down': 33,
 'trouser': 34,
 'midi': 35,
 'pant': 36,
 'legging': 37,
 'print': 38,
 'knit': 39,
 'culotte': 40,
 'sweatshirt': 41,
 'peacoat': 42,
 'kaftan': 43,
 'overalls': 44,
 'jogger': 45,
 'tee': 46,
 'combo': 47,
 'henley': 48,
 'cami': 49,
 'blouson': 50,
 'turtleneck': 51,
 'trousers': 52,
 'overcoat': 53,
 'hoodie': 54,
 't-shirt': 55,
 'caftan': 56,
 'tight': 57,
 'kimono': 58,
 'for': 59,
 'crewneck': 60,
 'skirts': 61,
 'parka': 62,
 'buttondown': 63,
 'skort': 64,
 'sweatershirt': 65,
 'sweatpants': 66,

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

# Example input and output data
# input_texts = [data['review_text'] for data in ori_dataset]
# output_labels = [data['category'] for data in ori_dataset]

label_map={}
cnt=0
for data in thedic:
  #if thedic[data]>20 and thedic[data]<300:
    label_map[data]=cnt
    cnt+=1
input_texts = [data['review_text'] for data in ori_dataset if data['category'] in label_map]
output_labels = [data['category'] for data in ori_dataset if data['category'] in label_map]

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=68)

# Tokenize and encode the input texts
input_ids = []
attention_masks = []
for input_text in input_texts:
    inputs = tokenizer.encode_plus(
        input_text,
        padding='longest',
        truncation=True,
        max_length=128,  # Replace with your desired maximum sequence length
        return_tensors='pt'
    )
    input_ids.append(inputs.input_ids.squeeze())
    attention_masks.append(inputs.attention_mask.squeeze())

# Pad the input sequences to ensure equal length
input_ids = pad_sequence(input_ids, batch_first=True)
attention_masks = pad_sequence(attention_masks, batch_first=True)

# Convert the output labels to a tensor
#label_map = {"dress": 0, "movie": 1}  # Define your label mapping

labels = torch.tensor([label_map[label] for label in output_labels])

# Create a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Define training hyperparameters
batch_size = 32
num_epochs = 4
learning_rate = 1e-4

# Create a DataLoader for batched training
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
  with tqdm(total=len(data_loader),unit='batch') as pbar:
    for batch in data_loader:
        batch_input_ids, batch_attention_masks, batch_labels = batch
        batch_input_ids = batch_input_ids.to(device)
        batch_attention_masks = batch_attention_masks.to(device)
        batch_labels = batch_labels.to(device)

        # Forward pass
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)

        # Backward pass and optimization
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item())
        pbar.update(1)

# Prediction example
test_input = "Ilove this dress so damn ."
inputs = tokenizer.encode_plus(
    test_input,
    padding='longest',
    truncation=True,
    max_length=128,  # Replace with your desired maximum sequence length
    return_tensors='pt'
)
input_ids = inputs.input_ids.squeeze().to(device)
attention_mask = inputs.attention_mask.squeeze().to(device)

with torch.no_grad():
    outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))

predicted_label_id = torch.argmax(outputs.logits, dim=1).item()
predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_label_id)]

print("Predicted label:", predicted_label)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Predicted label: dress


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
def predict(text_input,model,label_map,tokenizer=tokenizer,device=device):
  # Prediction example
  test_input = text_input
  inputs = tokenizer.encode_plus(
      test_input,
      padding='longest',
      truncation=True,
      max_length=128,  # Replace with your desired maximum sequence length
      return_tensors='pt'
  )
  input_ids = inputs.input_ids.squeeze().to(device)
  attention_mask = inputs.attention_mask.squeeze().to(device)

  with torch.no_grad():
      outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))

  predicted_label_id = torch.argmax(outputs.logits, dim=1).item()
  predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_label_id)]

  #print("Predicted label:", predicted_label)
  return predicted_label

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
acc=0
all=0
with tqdm(total=len(input_texts),unit='loop') as pbar:
  for i in range(len(input_texts)):

    pl=predict(input_texts[i],model,label_map)
    #print(output_labels[i])
    if pl==output_labels[i]:
      acc+=1
    all+=1
    pbar.update(1)
acc/=all
acc

100%|██████████| 2867/2867 [00:37<00:00, 77.38loop/s]


0.992675270317405

accuracy based on main 31 classes

In [ ]:
acc=0
all=0
with tqdm(total=len(input_texts),unit='loop') as pbar:
  for i in range(len(input_texts)):

    pl=predict(input_texts[i],model,label_map)
    #print(output_labels[i])
    if pl==output_labels[i]:
      acc+=1
    all+=1
    pbar.update(1)
acc/=all
acc


100%|██████████| 191667/191667 [45:32<00:00, 70.15loop/s]


0.6751136084980722

accuracy based on all classes

In [ ]:
acc=0
all=0
model_ori= BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/classification1.bin')
model_ori.to(device)
input_texts = [data['review_text'] for data in ori_dataset]
output_labels = [data['category'] for data in ori_dataset]
label_map={}
cnt=0
for data in thedic:

    label_map[data]=cnt
    cnt+=1
with tqdm(total=len(input_texts),unit='loop') as pbar:
  for i in range(len(input_texts)):

    pl=predict(input_texts[i],model_ori,label_map)
    #print(output_labels[i])
    if pl==output_labels[i]:
      acc+=1
    all+=1
    pbar.update(1)
acc/=all
acc

100%|██████████| 192544/192544 [41:57<00:00, 76.49loop/s]


0.6751028336380256

In [ ]:
acc=0
all=0
with tqdm(total=len(input_texts),unit='loop') as pbar:
  for i in range(len(input_texts)):

    pl=predict(input_texts[i],model,label_map)
    #print(output_labels[i])
    if pl==output_labels[i]:
      acc+=1
    all+=1
    pbar.update(1)
acc/=all
acc

100%|██████████| 192544/192544 [44:34<00:00, 72.00loop/s]


0.48240402193784276

In [ ]:
thedic['dress']/len(input_texts)

0.48240402193784276

In [ ]:
model.save_pretrained('/content/drive/MyDrive/CSE256_summary/models/classification_ori_2.bin')

Summarizer

In [ ]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model=Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text='OpenGenus Foundation is an open-source non-profit organization with the aim to enable people to work offline for a longer stretch, reduce the time spent on searching by exploiting the fact that almost 90% of the searches are same for every generation and to make programming more accessible.OpenGenus is all about positivity and innovation.Over 1000 people have contributed to our missions and joined our family. We have been sponsored by three great companies namely Discourse, GitHub and DigitalOcean. We run one of the most popular Internship program and open-source projects and have made a positive impact over people '
summary=model(text,ratio=0.2)
print(summary)

OpenGenus Foundation is an open-source non-profit organization with the aim to enable people to work offline for a longer stretch, reduce the time spent on searching by exploiting the fact that almost 90% of the searches are same for every generation and to make programming more accessible. OpenGenus is all about positivity and innovation.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
raw_reviews = []
raw_summaries = []

for each_data in ori_dataset:
  # if ('reviewText' not in each_data) or ('summary' not in each_data):
  #   continue
  raw_reviews.append(each_data['review_text'])
  raw_summaries.append(each_data['review_summary'])

In [6]:
punctuations = set(string.punctuation)

reviews = []
summaries = []

for re in raw_reviews:
  re=''.join([c for c in re.lower() if not c in punctuations])
  reviews.append(re)

for re in raw_summaries:
  re=''.join([c for c in re.lower() if not c in punctuations])
  summaries.append(re)

In [ ]:
print(reviews[0])
print(summaries[0])
test = reviews[0]
print(model(test))

an adorable romper belt and zipper were a little hard to navigate in a full day of wearbathroom use but thats to be expected wish it had pockets but other than that absolutely perfect i got a million compliments
so many compliments
an adorable romper belt and zipper were a little hard to navigate in a full day of wearbathroom use but thats to be expected wish it had pockets but other than that absolutely perfect i got a million compliments


In [ ]:
from rouge import Rouge
import random
from tqdm import tqdm

In [ ]:
test_reviews = reviews[:]
random.shuffle(test_reviews)
test_reviews = test_reviews[:4805]
test_summaries = summaries[:]
random.shuffle(test_summaries)
test_summaries = test_summaries[:4805]

def compute_rouge(model,test_reviews=test_reviews,test_summaries=test_summaries):
  scores = []
  rouger = Rouge()

  with tqdm(total = 1000) as pbar:
      for i in range(1000):
        if model(test_reviews[i])=="" or test_summaries[i]=="":
          pbar.update(1)
          continue
        score = rouger.get_scores(model(test_reviews[i]), test_summaries[i])
        score_1 = score[0]['rouge-1']
        score_2 = score[0]['rouge-2']
        score_L = score[0]['rouge-l']
        scores.append([score_1,score_2,score_L])
        pbar.update(1)

  score_1_list=[0,0,0]
  score_2_list=[0,0,0]
  score_L_list=[0,0,0]
  with tqdm(total = len(scores)) as pbar:
      for i in range(len(scores)):

        score_1 = scores[i][0]
        score_2 = scores[i][1]
        score_L = scores[i][2]
        score_1_list = [score_1_list[0]+score_1['r'],score_1_list[1]+score_1['p'],score_1_list[2]+score_1['f']]
        score_2_list = [score_2_list[0]+score_2['r'],score_2_list[1]+score_2['p'],score_2_list[2]+score_2['f']]
        score_L_list = [score_L_list[0]+score_L['r'],score_L_list[1]+score_L['p'],score_L_list[2]+score_L['f']]

        pbar.update(1)

  score_1_list=[ele/len(scores) for ele in score_1_list]
  score_2_list=[ele/len(scores) for ele in score_2_list]
  score_L_list=[ele/len(scores) for ele in score_L_list]

  return [score_1_list,score_2_list,score_L_list]

In [ ]:
res = compute_rouge(model)
res

100%|██████████| 865/865 [00:00<00:00, 505725.25it/s]


[[0.2575494558194678, 0.0507137698787139, 0.0778330143813534],
 [0.026028211296290526, 0.0038663408059063033, 0.006122858016242947],
 [0.22011091036377828, 0.0416925227610884, 0.0642301204844638]]

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
print(reviews[1])
print(summaries[1])
test = raw_reviews[1]
print(summarizer(raw_reviews[1],min_length=0,max_length=31))

i rented this dress for a photo shoot the theme was hollywood glam and big beautiful hats the dress was very comfortable and easy to move around in it is definitely on my list to rent again for another formal event 
i felt so glamourous
[{'summary_text': ' The dress was very comfortable and easy to move around in . It is definitely on my list to rent again for another formal event .'}]


In [ ]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
def compute_rouge(model,test_reviews=test_reviews,test_summaries=test_summaries):
  scores = []
  rouger = Rouge()

  with tqdm(total = 1000) as pbar:
      for i in range(1000):
        if model(test_reviews[i])=="" or test_summaries[i]=="":
          pbar.update(1)
          continue
        score = rouger.get_scores(model(test_reviews[i],min_length=0,max_length=20)[0]['summary_text'], test_summaries[i])
        score_1 = score[0]['rouge-1']
        score_2 = score[0]['rouge-2']
        score_L = score[0]['rouge-l']
        scores.append([score_1,score_2,score_L])
        pbar.update(1)

  score_1_list=[0,0,0]
  score_2_list=[0,0,0]
  score_L_list=[0,0,0]
  with tqdm(total = len(scores)) as pbar:
      for i in range(len(scores)):

        score_1 = scores[i][0]
        score_2 = scores[i][1]
        score_L = scores[i][2]
        score_1_list = [score_1_list[0]+score_1['r'],score_1_list[1]+score_1['p'],score_1_list[2]+score_1['f']]
        score_2_list = [score_2_list[0]+score_2['r'],score_2_list[1]+score_2['p'],score_2_list[2]+score_2['f']]
        score_L_list = [score_L_list[0]+score_L['r'],score_L_list[1]+score_L['p'],score_L_list[2]+score_L['f']]

        pbar.update(1)

  score_1_list=[ele/len(scores) for ele in score_1_list]
  score_2_list=[ele/len(scores) for ele in score_2_list]
  score_L_list=[ele/len(scores) for ele in score_L_list]

  return [score_1_list,score_2_list,score_L_list]
res = compute_rouge(summarizer)
res

100%|██████████| 998/998 [00:00<00:00, 354288.23it/s]


[[0.1332809372135704, 0.06759873106905435, 0.0823906037977952],
 [0.008732189419880588, 0.004851461164086415, 0.005824475049595652],
 [0.11943553162232806, 0.058784540181157204, 0.07221230039589481]]

In [ ]:
long_reviews=[review for review in reviews if len(review)>220]
id=reviews.index(long_reviews[0])
long_reviews[0]

'i rented this for my companys black tie awards banquet  i liked that this dress was short but was a little fancier with the sequins i generally dont care for long dresses  i would describe the color as more rose gold than yellow gold  i have blonde hair and fair skin and the color was very flattering  this is a very forgiving dress its form fitting without making you feel like youre squeezed into it  i would rent this one again  highly recommend'

In [ ]:
summaries[id]

'dress arrived on time and in perfect condition '

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = 'facebook/bart-base'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def generate_summary(text):
    # Tokenize the input text
    input_ids = tokenizer.encode(text, truncation=True, padding='longest', return_tensors='pt')

    # Generate the summary
    summary_ids = model.generate(input_ids, num_beams=4, max_length=50, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

input_text = long_reviews[0]
summary = generate_summary(input_text)
print("Summary:", summary)

Summary: i rented this for my companys black tie awards banquet  i liked that this dress was short but was a little fancier with the sequins i generally dont care for long dresses  i would describe the color as more rose gold than


In [ ]:
pairs=[{'input_text':reviews[i],'target_text':summaries[i]} for i in range(len(reviews))]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length, max_target_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['input_text']
        target_text = item['target_text']

        # Tokenize and encode the input text
        input_encoding = self.tokenizer.encode_plus(
            input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_input_length,
            return_tensors='pt'
        )

        # Tokenize and encode the target text
        target_encoding = self.tokenizer.encode_plus(
            target_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_target_length,
            return_tensors='pt'
        )

        # Return the encoded inputs and targets as tensors
        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'target_ids': target_encoding['input_ids'].squeeze(),
            'target_attention_mask': target_encoding['attention_mask'].squeeze()
        }

# Define training hyperparameters
batch_size = 16
num_epochs = 1
learning_rate = 1e-6

# Load and preprocess your dataset
data = pairs  # Your dataset here
data = [da for da in data if len(da['input_text'].split())>50]
random.shuffle(data)
data = data[:int(len(data)*0.4)]

# Create data loader for batched training
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
dataset = CustomDataset(data=data, tokenizer=tokenizer, max_input_length=100, max_target_length=15)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
  with tqdm(total = len(data_loader), unit = 'batch') as pbar:
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_ids = batch['target_ids'].to(device)
        labels_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_ids)

        # Backward pass and optimization
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.update(1)
        pbar.set_postfix(loss=loss.item())
    # Evaluation on validation set
    # ...

# Save the trained model
model.save_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert5.bin')

100%|██████████| 2355/2355 [30:32<00:00,  1.28batch/s, loss=1.52]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
model3 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert3.bin')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3.to(device)

# Input text for summarization
input_text = reviews[10]

# Tokenize and encode the input text
input_encoding = tokenizer.encode_plus(
    input_text,
    padding='max_length',
    truncation=True,
    max_length=300,
    return_tensors='pt'
)

input_ids = input_encoding['input_ids'].to(device)
attention_mask = input_encoding['attention_mask'].to(device)

summary_ids = model3.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    min_length=20,  # Adjust the minimum length of the summary
    max_length=50,  # Adjust the maximum length of the summary
    num_beams=4,  # Adjust the number of beams for beam search
    early_stopping=True
)


# Decode the generated summary
summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
print("Generated Summary:", summary)

Generated Summary: i wore this dress to a wedding and it was the perfect dress for a night out


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

def predict_summary(model, input_text, tokenizer = tokenizer,device = device):
  # Set the device for inference
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)



  # Tokenize and encode the input text
  input_encoding = tokenizer.encode_plus(
      input_text,
      padding='max_length',
      truncation=True,
      max_length=300,
      return_tensors='pt'
  )

  input_ids = input_encoding['input_ids'].to(device)
  attention_mask = input_encoding['attention_mask'].to(device)

  summary_ids = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      min_length=20,  # Adjust the minimum length of the summary
      max_length=50,  # Adjust the maximum length of the summary
      num_beams=4,  # Adjust the number of beams for beam search
      early_stopping=True
  )


  # Decode the generated summary
  summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  #print("Generated Summary:", summary)
  return summary

In [ ]:
def compute_rouge(model,test_reviews=test_reviews,test_summaries=test_summaries):
  scores = []
  rouger = Rouge()

  with tqdm(total = 1000) as pbar:
      for i in range(1000):
        if test_reviews[i]=="" or test_summaries[i]=="":
          pbar.update(1)
          continue
        score = rouger.get_scores(predict_summary(model, test_reviews[i]), test_summaries[i])
        score_1 = score[0]['rouge-1']
        score_2 = score[0]['rouge-2']
        score_L = score[0]['rouge-l']
        scores.append([score_1,score_2,score_L])
        pbar.update(1)

  score_1_list=[0,0,0]
  score_2_list=[0,0,0]
  score_L_list=[0,0,0]
  with tqdm(total = len(scores)) as pbar:
      for i in range(len(scores)):

        score_1 = scores[i][0]
        score_2 = scores[i][1]
        score_L = scores[i][2]
        score_1_list = [score_1_list[0]+score_1['r'],score_1_list[1]+score_1['p'],score_1_list[2]+score_1['f']]
        score_2_list = [score_2_list[0]+score_2['r'],score_2_list[1]+score_2['p'],score_2_list[2]+score_2['f']]
        score_L_list = [score_L_list[0]+score_L['r'],score_L_list[1]+score_L['p'],score_L_list[2]+score_L['f']]

        pbar.update(1)

  score_1_list=[ele/len(scores) for ele in score_1_list]
  score_2_list=[ele/len(scores) for ele in score_2_list]
  score_L_list=[ele/len(scores) for ele in score_L_list]

  return [score_1_list,score_2_list,score_L_list]

In [ ]:
model3 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert3.bin')
model3.eval()
model3.to(device)

res = compute_rouge(model3)
res

100%|██████████| 998/998 [00:00<00:00, 443047.78it/s]


[[0.1964289028494065, 0.10061609593579908, 0.12116461197963825],
 [0.02723284983273224, 0.013132419111366155, 0.015927334138343567],
 [0.17651086093287835, 0.08659989255828797, 0.10556838204478008]]

In [ ]:
model4 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert4.bin')
model4.eval()
model4.to(device)

res = compute_rouge(model4)
res

100%|██████████| 998/998 [00:00<00:00, 352438.78it/s]


[[0.2088669853126788, 0.10510729261900363, 0.1278079790761505],
 [0.029311578593133165, 0.015564140835852974, 0.018573061122216725],
 [0.18533640805266086, 0.08907922700620362, 0.1098615471716593]]

In [ ]:
model5 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert5.bin')
model5.eval()
model5.to(device)

res = compute_rouge(model5)
res

100%|██████████| 998/998 [00:00<00:00, 343023.47it/s]


[[0.21494241288078217, 0.11053944508279685, 0.1336255052986164],
 [0.030278595297138555, 0.01660860751100052, 0.019786722744914016],
 [0.1904594470136544, 0.09380288616186919, 0.11491825449269089]]

In [ ]:
model_ori = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model_ori.eval()
model_ori.to(device)

res = compute_rouge(model_ori)
res

100%|██████████| 998/998 [00:00<00:00, 362103.41it/s]


[[0.222301670994839, 0.05817121760180825, 0.0850286227111279],
 [0.019558000745918378, 0.004979709725239653, 0.00733089206301588],
 [0.19856516411332692, 0.049843720425299824, 0.07337597828642953]]

In [ ]:
model1 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert.bin')
model1.eval()
model1.to(device)

res = compute_rouge(model1)
res

100%|██████████| 998/998 [00:00<00:00, 332083.73it/s]


[[0.24321684324511195, 0.10903056112224449, 0.13805731556156497],
 [0.058539313125497976, 0.02379759519038076, 0.02987108690585829],
 [0.21930219537923337, 0.09531563126252506, 0.1216900746735826]]

In [ ]:
model2 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert2.bin')
model2.eval()
model2.to(device)

res = compute_rouge(model2)
res

100%|██████████| 998/998 [00:00<00:00, 375351.09it/s]


[[0.10821708459375458, 0.09008016032064119, 0.09115203768761816],
 [0.009331532387812288, 0.009018036072144285, 0.008679157664986881],
 [0.10315851225359898, 0.0835671342685371, 0.08559694778490755]]

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CSE256_summary/models/bert.bin')
model1.eval()
model1.to(device)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')



# Input text for summarization
input_text = reviews[10]

# Tokenize and encode the input text
input_encoding = tokenizer.encode_plus(
    input_text,
    padding='max_length',
    truncation=True,
    max_length=300,
    return_tensors='pt'
)

input_ids = input_encoding['input_ids'].to(device)
attention_mask = input_encoding['attention_mask'].to(device)

summary_ids = model1.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    min_length=20,  # Adjust the minimum length of the summary
    max_length=50,  # Adjust the maximum length of the summary
    num_beams=4,  # Adjust the number of beams for beam search
    early_stopping=True
)


# Decode the generated summary
summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
print("Generated Summary:", summary)

Generated Summary: i wore this dress for my bridal shower and absolutely loved it i got so many compliments 


In [ ]:
for i in range(10):
  predict_summary(model, long_reviews[i])

  print(summaries[i])
  print('')

Generated Summary: i rented this dress for a black tie awards banquet and it was very flattering 
so many compliments

Generated Summary: i wore this for a work event and it was a great fit for me and my
i felt so glamourous

Generated Summary: i received tons of compliments on this dress and it was perfect for a black tie wedding
it was a great time to celebrate the almost completion of my first year of law school

Generated Summary: i felt like a million bucks in this dress i was able to dance the night away
dress arrived on time and in perfect condition 

Generated Summary: gorgeous and sparkly  perfect for a black tie nye nye wedding
was in love with this dress 

Generated Summary: i felt like a million bucks in this dress it was perfect for my bacheloret
traditional with a touch a sass

Generated Summary: i wore this dress to a wedding and it was a great dress for the event 
literally the cutest dress ever

Generated Summary: i felt like a million bucks in this dress it was perfec